In [37]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy
from src.models import TwoLayerLeaky
from src.train_model import training_loop

In [38]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [39]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

In [40]:
batch_size = 64
dtype = torch.float

## Data

In [41]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [42]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [43]:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

## Prepare model

In [44]:
model = TwoLayerLeaky(num_inputs=8, num_hidden=1200, num_outputs=1, beta=0.95, num_steps=20)
print(model)

TwoLayerLeaky(
  (fc1): Linear(in_features=8, out_features=1200, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=1200, out_features=1, bias=True)
)


In [45]:
training_loop(model, train_loader, valid_loader, device, num_epochs=30)

100%|██████████| 30/30 [00:51<00:00,  1.73s/it]

Accuracy on validation dataset: 36.06936416184971%


In [46]:
get_accuracy(model, test_loader, device=device, pct_close=0.2)

0.38670520231213873